In [1]:
import findspark

In [2]:

findspark.init("/opt/manual/spark")

In [3]:

from pyspark.sql import SparkSession, functions as F
import pandas as pd

In [4]:
spark = SparkSession.builder \
.appName("arabam_com") \
.master("yarn") \
.getOrCreate()

In [5]:
! hdfs dfs -rm /user/train/datasets/arabam.csv

Deleted /user/train/datasets/arabam.csv


In [6]:
! hdfs dfs -put ~/Desktop/arabam/arabam.csv /user/train/datasets/

In [7]:
df1 = spark.read.format("csv") \
.option("header",True) \
.option("inferShema",True) \
.option("sep",",")  \
.load("hdfs://localhost:9000/user/train/datasets/arabam.csv")

In [8]:
df1.limit(10).toPandas()

,_c0,İlan No:,İlan Tarihi:,Marka:,Seri:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Motor Hacmi:,Motor Gücü:,Kilometre:,car_link:,price
0,0,16251517,23 Aralık 2020,Volkswagen,Passat Variant,1.6 TDi BlueMotion Comfortline,2012,Dizel,Yarı Otomatik,1598 cc,105 hp,70.000 km,https://www.arabam.com//ilan/sahibinden-satili...,200.000 TL
1,1,15943402,22 Aralık 2020,Volkswagen,Polo,1.4 Comfortline,2013,LPG & Benzin,Düz,1390 cc,85 hp,127.000 km,https://www.arabam.com//ilan/galeriden-satilik...,122.500 TL
2,2,15788492,22 Aralık 2020,Chevrolet,Cruze,1.6 LS,2009,LPG & Benzin,Düz,1598 cc,113 hp,226.000 km,https://www.arabam.com//ilan/galeriden-satilik...,85.000 TL
3,3,16250683,22 Aralık 2020,Renault,Megane,1.5 dCi Touch,2020,Dizel,Yarı Otomatik,1461 cc,115 hp,0 km,https://www.arabam.com//ilan/galeriden-satilik...,277.000 TL
4,4,16250505,22 Aralık 2020,Volkswagen,VW CC,1.4 TSi,2013,Benzin,Yarı Otomatik,1390 cc,160 hp,49.000 km,https://www.arabam.com//ilan/galeriden-satilik...,192.500 TL
5,5,15793297,22 Aralık 2020,Renault,Clio,1.5 dCi Touch,2017,Dizel,Yarı Otomatik,1461 cc,90 hp,80.000 km,https://www.arabam.com//ilan/galeriden-satilik...,154.900 TL
6,6,16249278,22 Aralık 2020,Fiat,Linea,1.3 Multijet Active Plus,2011,Dizel,Düz,1248 cc,90 hp,109.900 km,https://www.arabam.com//ilan/sahibinden-satili...,85.000 TL
7,7,16248951,22 Aralık 2020,Seat,Ibiza,1.0 EcoTSI Style,2017,Benzin,Düz,999 cc,95 hp,37.500 km,https://www.arabam.com//ilan/sahibinden-satili...,155.000 TL
8,8,15293099,22 Aralık 2020,Volkswagen,Passat,1.6 TDi BlueMotion Highline,2015,Dizel,Yarı Otomatik,1598 cc,120 hp,113.000 km,https://www.arabam.com//ilan/galeriden-satilik...,365.000 TL
9,9,16097555,22 Aralık 2020,Volkswagen,Jetta,1.6 TDi Comfortline,2011,Dizel,Yarı Otomatik,1598 cc,105 hp,188.000 km,https://www.arabam.com//ilan/galeriden-satilik...,151.500 TL


In [9]:
df1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- İlan No:: string (nullable = true)
 |-- İlan Tarihi:: string (nullable = true)
 |-- Marka:: string (nullable = true)
 |-- Seri:: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- Yıl:: string (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- Motor Hacmi:: string (nullable = true)
 |-- Motor Gücü:: string (nullable = true)
 |-- Kilometre:: string (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- price: string (nullable = true)



In [10]:
df1.columns

['_c0',
 'İlan No:',
 'İlan Tarihi:',
 'Marka:',
 'Seri:',
 'Model:',
 'Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'Motor Hacmi:',
 'Motor Gücü:',
 'Kilometre:',
 'car_link:',
 'price']

In [11]:
for col in df1.columns:
    df1 = df1.withColumn(col, F.ltrim(F.rtrim(df1[col])))

In [142]:
df2 =df1.withColumn("motor_hacmi",F.split('Motor Hacmi:',"-")[0]) \
.withColumnn("price",F.split("price"," ")[0])
.withColumn("Kilometre:",F.split("Kilometre:"," ")[0]) \
.drop(F.col('Motor Hacmi:')) 

 


In [148]:
df3 =df2.withColumn("motor_hacmi",F.split("motor_hacmi"," ")[0]) \
.withColumn("motor_gucu",F.split("Motor Gücü:"," ")[0]) \
.withColumn("kilometre",F.split("Kilometre:",".")[0][1]) \
.withColumn("marka",F.concat(F.col("Marka:"),F.lit('_'),F.col("Seri:"))) \
.drop(F.col("Kilometre:")) \
.drop(F.col("Motor Gücü:")) \
.drop(F.col("Marka:")) \
.drop(F.col("Seri:")) \
.drop(F.col("_c0")) \
.drop(F.col("İlan Tarihi:"))

In [149]:
df3.limit(2).toPandas()

,İlan No:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,car_link:,motor_hacmi,motor_gucu,kilometre,marka
0,16224840,1.6 TDi BlueMotion Highline,2017,Dizel,Yarı Otomatik,https://www.arabam.com//ilan/galeriden-satilik...,1598,120,63.000,Volkswagen_Passat
1,16224785,530xd Gran Turismo Premium,2011,Dizel,Yarı Otomatik,https://www.arabam.com//ilan/galeriden-satilik...,2501,226,350.000,BMW_5 Serisi


In [150]:
df3.printSchema()

root
 |-- İlan No:: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- Yıl:: string (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- motor_hacmi: string (nullable = true)
 |-- motor_gucu: string (nullable = true)
 |-- kilometre: string (nullable = true)
 |-- marka: string (nullable = true)



In [151]:
df3.columns

['İlan No:',
 'Model:',
 'Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'car_link:',
 'motor_hacmi',
 'motor_gucu',
 'kilometre',
 'marka']

In [168]:
df4=df3.select(
F.col('Yıl:').cast("int"),
F.col('Yakıt Tipi:'),
F.col('Vites Tipi:'),
F.col('motor_hacmi').cast("int"),
F.col('motor_gucu').cast("int"),
F.col('kilometre').cast("int"),
F.col('marka'),
F.col('Model:'))

In [169]:
df4.printSchema()

root
 |-- Yıl:: integer (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- motor_hacmi: integer (nullable = true)
 |-- motor_gucu: integer (nullable = true)
 |-- kilometre: integer (nullable = true)
 |-- marka: string (nullable = true)
 |-- Model:: string (nullable = true)



In [170]:
df4.limit(3).toPandas()

,Yıl:,Yakıt Tipi:,Vites Tipi:,motor_hacmi,motor_gucu,kilometre,marka,Model:
0,2017,Dizel,Yarı Otomatik,1598,120,63,Volkswagen_Passat,1.6 TDi BlueMotion Highline
1,2011,Dizel,Yarı Otomatik,2501,226,350,BMW_5 Serisi,530xd Gran Turismo Premium
2,1999,LPG & Benzin,Otomatik,1598,107,163,Renault_Megane,2.0 RXE


In [171]:
# Null check
for(col_name, col_type) in zip(df4.columns, df4.dtypes):
    null_count = df4.filter( (F.col(col_name).isNull()) | (F.col(col_name) == "")).count()
    if(  null_count > 0 ):
        print("{} {} type has {} null values".format(col_name, col_type[1], null_count))

Yıl: int type has 1 null values
Yakıt Tipi: string type has 1 null values
motor_hacmi int type has 13 null values
motor_gucu int type has 20 null values


In [172]:
yil_na=df4.filter(F.col("Yıl:").isNull()).select("marka").collect()[0][0]
yil_na1=df4.filter(F.col("marka")==yil_na).select("Yıl:","marka").groupBy("marka").avg("Yıl:").collect()[0][1]
yil_na1=int(yil_na1)
yil_na1

2009

In [173]:
df5=df4.na.fill(value=yil_na1,subset=["Yıl:"])

In [174]:
for(col_name, col_type) in zip(df5.columns, df5.dtypes):
    null_count = df5.filter( (F.col(col_name).isNull()) | (F.col(col_name) == "")).count()
    if(  null_count > 0 ):
        print("{} {} type has {} null values".format(col_name, col_type[1], null_count))

Yakıt Tipi: string type has 1 null values
motor_hacmi int type has 13 null values
motor_gucu int type has 20 null values


In [175]:
df5.filter(F.col("Yakıt Tipi:").isNull()).select("marka").collect()[0]

Row(marka='Toyota_Corolla')

In [176]:
df6=df5.na.fill(value="Benzin",subset="Yakıt Tipi:")

In [177]:
df6.limit(10).toPandas()

,Yıl:,Yakıt Tipi:,Vites Tipi:,motor_hacmi,motor_gucu,kilometre,marka,Model:
0,2017,Dizel,Yarı Otomatik,1598,120,63,Volkswagen_Passat,1.6 TDi BlueMotion Highline
1,2011,Dizel,Yarı Otomatik,2501,226,350,BMW_5 Serisi,530xd Gran Turismo Premium
2,1999,LPG & Benzin,Otomatik,1598,107,163,Renault_Megane,2.0 RXE
3,2017,Dizel,Düz,1248,95,105,Fiat_Egea,1.3 Multijet Urban
4,2013,Dizel,Yarı Otomatik,1598,90,82,Volkswagen_Polo,1.6 TDi Comfortline
5,2014,Dizel,Düz,1461,90,54,Dacia_Sandero,1.5 dCi Stepway
6,2017,Dizel,Yarı Otomatik,1598,120,105,Volkswagen_Passat Variant,1.6 TDi BlueMotion Comfortline
7,2017,Dizel,Düz,1560,93,93,Peugeot_301,1.6 HDi Active
8,2006,Benzin,Otomatik,1598,115,310,Volkswagen_Passat,1.6 FSi Comfortline
9,2004,LPG & Benzin,Düz,1598,115,235,Renault_Megane,1.6 Dynamique


In [178]:
df6.filter(F.col("motor_hacmi").isNull()).select("marka","motor_hacmi").toPandas()

,marka,motor_hacmi
0,Renault_R 19,NaN
1,Opel_Vectra,NaN
2,Audi_A3,NaN
3,Renault_Clio,NaN
4,Mercedes - Benz_200,NaN
5,Fiat_Linea,NaN
6,Seat_Leon,NaN
7,Fiat_Uno,NaN
8,Renault_R 12,NaN
9,Hyundai_Accent,NaN


In [183]:
df7=df6.withColumn("motor_hacmi",F.when((df6.marka=="Renault_R 19_1.6 Europa RTE") & (F.col("motor_hacmi").isNull()),1600) \
.when((df6.marka=="Renault_R 19_1.6 Europa RTE")&(F.col("motor_hacmi").isNull()),1600) \
.when((df6.marka=="Opel_Vectra_2.5 CDX")&F.col("motor_hacmi").isNull(),2500) \
.when((df6.marka=="Audi_A3_A3 Sportback 1.6 Ambition")&F.col("motor_hacmi").isNull(),1600) \
.when((df6.marka=="Renault_Clio_2.0 Williams")&F.col("motor_hacmi").isNull(),2000) \
.when((df6.marka=="Mercedes - Benz_200_D")&F.col("motor_hacmi").isNull(),2000) \
.when((df6.marka=="Fiat_Linea_1.3 Multijet Pop")&F.col("motor_hacmi").isNull(),1300) \
.when((df6.marka=="Seat_Leon_1.4 TSI Stylance")&F.col("motor_hacmi").isNull(),1400) \
.when((df6.marka=="Fiat_Uno_1.4 ie SX")&F.col("motor_hacmi").isNull(),1400) \
.when((df6.marka=="Renault_R 12_GTS")&F.col("motor_hacmi").isNull(),1600) \
.when((df6.marka=="Hyundai_Accent_1.3 LS")&F.col("motor_hacmi").isNull(),1300) \
.when((df6.marka=="Tofaş_Doğan_SLX ie")&F.col("motor_hacmi").isNull(),1600) \
.when((df6.marka=="Honda_Civic_1.6 i-VTEC ES")&F.col("motor_hacmi").isNull(),1600) \
.when((df6.marka=="Hyundai_i30_1.4 CVVT Team")&F.col("motor_hacmi").isNull(),1400).otherwise(F.col("motor_hacmi")))

In [184]:
for(col_name, col_type) in zip(df7.columns, df7.dtypes):
    null_count = df7.filter( (F.col(col_name).isNull()) | (F.col(col_name) == "")).count()
    if(  null_count > 0 ):
        print("{} {} type has {} null values".format(col_name, col_type[1], null_count))

motor_hacmi int type has 13 null values
motor_gucu int type has 20 null values


In [186]:
df8=df7.dropna(how="any")

In [187]:

categoric_cols = []
numeric_cols = []
discarted_cols = []
label_col = []

In [188]:
# column seperation
for col_name in df7.dtypes:
    if (col_name[0] not in []):
        if (col_name[1] == 'string'):
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])

In [189]:
for col_name in categoric_cols:
    df7.groupBy(col_name).agg(F.count("*").alias("TotalCount")).orderBy(F.desc("TotalCount")).show()

+------------+----------+
| Yakıt Tipi:|TotalCount|
+------------+----------+
|       Dizel|       443|
|LPG & Benzin|       312|
|      Benzin|       211|
+------------+----------+

+-------------+----------+
|  Vites Tipi:|TotalCount|
+-------------+----------+
|          Düz|       636|
|Yarı Otomatik|       186|
|     Otomatik|       144|
+-------------+----------+

+-----------------+----------+
|            marka|TotalCount|
+-----------------+----------+
|     Renault_Clio|        54|
|       Opel_Astra|        52|
|Volkswagen_Passat|        47|
|   Toyota_Corolla|        41|
|   Renault_Megane|        38|
|       Ford_Focus|        37|
|       Fiat_Linea|        34|
|  Volkswagen_Polo|        31|
|  Renault_Fluence|        30|
|  Volkswagen_Golf|        24|
|      Honda_Civic|        22|
|   Renault_Symbol|        21|
| Volkswagen_Jetta|        21|
|        Fiat_Egea|        19|
|      Ford_Fiesta|        18|
|      Tofaş_Şahin|        17|
|       Opel_Corsa|        14|
|      

In [212]:
df7.columns

['Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'motor_hacmi',
 'motor_gucu',
 'kilometre',
 'marka',
 'Model:']

In [ ]:

# column seperation after analysis
categoric_cols = []
numeric_cols = []
# Why we discard these? Because they have high cardinality. In other words they have too many distinct categories
# Adding them to model will not do good if not too bad.
discarted_cols = ['CustomerID', 'ServiceArea']
label_col = ['Churn']

In [ ]:

for col_name in df1.dtypes:
    if (col_name[0] not in discarted_cols+label_col):
        if (col_name[1] == 'string'):
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])

In [195]:
df7.select("marka").groupBy("marka").agg(F.count("*")).toPandas()["marka"].value_counts().sum()

144

In [197]:
# Here we count the distinct categories in categoric columns. If there is more than two,
# we will add those to to_be_onehotencoded_cols list If there is just 2 we don't need to use onehotencoder
# So if distinct category gt 2 we have to add it to_be_onehotencoded_cols list
to_be_onehotencoded_cols = []

for col_name in categoric_cols:
    count = df7.select(col_name).distinct().count()
    if count > 2:
        to_be_onehotencoded_cols.append(col_name)
    print("{} has {} distinct category.".format(col_name, count))
    

Yakıt Tipi: has 3 distinct category.
Vites Tipi: has 3 distinct category.
marka has 144 distinct category.
Model: has 488 distinct category.


In [198]:
print(to_be_onehotencoded_cols)

['Yakıt Tipi:', 'Vites Tipi:', 'marka', 'Model:']


# StringIndexer

In [199]:
from pyspark.ml.feature import StringIndexer

In [200]:
# Will hold stringIndexer objects and column names
my_dict = {}

# Will collect StringIndexer ojects
string_indexer_objs = []

# Will collect StringIndexer output colnames
string_indexer_output_names = []

# Will collect OneHotEncoder output colnames
ohe_col_input_names = []
ohe_col_output_names = []

for col_name in categoric_cols:
    my_dict[col_name+"_indexobj"] = StringIndexer() \
    .setHandleInvalid('skip') \
    .setInputCol(col_name) \
    .setOutputCol(col_name+"_indexed")
    
    string_indexer_objs.append(my_dict.get(col_name+"_indexobj"))
    string_indexer_output_names.append(col_name+"_indexed")
    if col_name in to_be_onehotencoded_cols:
        ohe_col_input_names.append(col_name+"_indexed")
        ohe_col_output_names.append(col_name+"_ohe")

In [201]:
print(string_indexer_objs)
print(len(string_indexer_objs))

[StringIndexer_bfcee90c195c, StringIndexer_42d2032fd146, StringIndexer_515f38b92be6, StringIndexer_bd1f2bb8a9f8]
4


In [203]:
print(ohe_col_input_names)
print(len(ohe_col_input_names))

['Yakıt Tipi:_indexed', 'Vites Tipi:_indexed', 'marka_indexed', 'Model:_indexed']
4


In [202]:
print(string_indexer_output_names)
print(len(string_indexer_output_names))

['Yakıt Tipi:_indexed', 'Vites Tipi:_indexed', 'marka_indexed', 'Model:_indexed']
4


# OnehotEncoder

In [204]:
from pyspark.ml.feature import OneHotEncoder

In [205]:
encoder = OneHotEncoder() \
.setInputCols(ohe_col_input_names) \
.setOutputCols(ohe_col_output_names)

In [206]:
from pyspark.ml.feature import VectorAssembler

## VectorAssembler

In [207]:
string_indexer_col_names_ohe_exluded = list(set(string_indexer_output_names).difference(set(ohe_col_input_names)))
string_indexer_col_names_ohe_exluded

[]

In [208]:
if len(categoric_cols) == (len(string_indexer_col_names_ohe_exluded)+len(ohe_col_output_names)):
    print("Columns verified")
else: print("Columns are not verified")

Columns verified


In [209]:
assembler = VectorAssembler().setHandleInvalid("skip") \
.setInputCols(numeric_cols+string_indexer_col_names_ohe_exluded+ohe_col_output_names) \
.setOutputCol('unscaled_features')

In [211]:
label_col

[]

In [210]:
from pyspark.ml.feature import StandardScaler

IndexError: list index out of range

In [ ]:
scaler = StandardScaler().setInputCol("unscaled_features").setOutputCol("scaled_features")

## PCA

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
#pca = PCA().setInputCol("scaled_features").setK(10).setOutputCol("features")

# Estimator: GBTClassifier

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
estimator = GBTClassifier() \
.setFeaturesCol("features") \
.setLabelCol("label")

## Pipeline

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline_obj = Pipeline().setStages(string_indexer_objs+[encoder, assembler, label_indexer, scaler, pca, estimator])

# Split data

In [ ]:
train_df, test_df = df1.randomSplit([0.2,0.8], seed=142)

# Train MoDEL

In [ ]:
pipeline_model = pipeline_obj.fit(train_df)

# Save the model

In [ ]:
pipeline_model.stages[-1].write().overwrite().save("saved_models/gbt_churn_pca10")

# Prediction

In [ ]:
transformed_df = pipeline_model.transform(test_df)

In [ ]:
transformed_df.select('label','prediction','probability','rawPrediction').show(truncate=False)

# Evaluate the model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator()

In [ ]:
evaluator.evaluate(transformed_df)

In [ ]:
evaluator.getMetricName()

In [ ]:
transformed_df.select('label','prediction','probability','rawPrediction') \
.filter("prediction == 1.0") \
.show(truncate=False)

In [ ]:
transformed_df.select('label','prediction','probability','rawPrediction') \
.withColumn("score", F.col('probability').cast('string')).show()

In [ ]:
transformed_df.select('label','prediction','probability','rawPrediction').printSchema()

# Hyperparameter Tuning

In [ ]:
from pyspark.ml.tuning import CrossValidatorModel, ParamGridBuilder, CrossValidator

In [ ]:
paramGrid = ParamGridBuilder() \
.addGrid(pca.k, [15, 25]) \
.addGrid(estimator.maxDepth, [5, 10]) \
.addGrid(estimator.maxBins, [32, 50]) \
.build()

In [ ]:
crossval = CrossValidator(estimator=pipeline_obj,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5) 

In [ ]:
cvModel = crossval.fit(df1)

In [ ]:
type(cvModel)

In [ ]:
cvModel.params

In [ ]:
bestModel = cvModel.bestModel

In [ ]:
evaluator.evaluate(bestModel.transform(df1))

In [ ]:
evaluator.evaluate(cvModel.transform(df1))

In [ ]:
spark.stop()